## **silver-getting-historical-data**
<p>This notebook extracts raw historical stock price data—such as open, high, low, close, and volume—for multiple tickers from Yahoo Finance. It serves as the base time-series dataset for price trend, volatility, or return analysis in the silver and gold layers.</p>

#### **Importing necessary libraries**

In [7]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable
import datetime
import traceback

StatementMeta(, 38f1cb93-2571-4611-a6aa-60debdba0ef9, 10, Finished, Available, Finished)

#### **Variables**

In [8]:
file_name = "yfinance_data"
layer_name = "bronze"

StatementMeta(, 38f1cb93-2571-4611-a6aa-60debdba0ef9, 11, Finished, Available, Finished)

#### **Step 2: Read the Stock Historical Data and Breakdown Categories Data**

In [9]:
yf_hist_df = spark.read.parquet(f"Files/{layer_name}/{file_name}/")

StatementMeta(, 38f1cb93-2571-4611-a6aa-60debdba0ef9, 12, Finished, Available, Finished)

#### **Step 3: Select the necessary columns from the Historical Data**

In [10]:
output_df = yf_hist_df\
                        .select(
                            col('Company'),
                            col('Date'),
                            col('Open'),
                            col('High'),
                            col('Close'),
                            col('Low'),
                            col('Volume')
                        )

StatementMeta(, 38f1cb93-2571-4611-a6aa-60debdba0ef9, 13, Finished, Available, Finished)

#### **Step 4: Storing the transformed data into a Delta table in the Silver layer**

In [11]:
delta_path = f"Files/silver/{file_name}"

try:
    if DeltaTable.isDeltaTable(spark, delta_path):
        delta_table = DeltaTable.forPath(spark, delta_path)
        
        latest_date = delta_table.toDF().agg({"Date": "max"}).collect()[0][0]
        new_data_to_append = output_df.filter(col("Date") > latest_date)
        
        delta_table.alias("target").merge(
            new_data_to_append.alias("source"),
            (col("target.Company") == col("source.Company")) &
            (col("target.Date") == col("source.Date"))
        ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

    else:
        output_df.write.format("delta").mode("overwrite").save(delta_path)

except Exception as e:
    print("An error occurred during the processing:")
    traceback.print_exc()

StatementMeta(, 38f1cb93-2571-4611-a6aa-60debdba0ef9, 14, Finished, Available, Finished)